In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv("../input/train.csv")
X_train = train_df["question_text"].fillna("zxc").values
test_df = pd.read_csv("../input/test.csv")
X_test = test_df["question_text"].fillna("zxc").values
y = train_df["target"]

train_df.head()

In [ ]:
# Wow, people can be awful :(
train_df[train_df.target == 1].head()

In [ ]:
# Naive Bayes

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV

kf = StratifiedKFold(n_splits=3)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', MultinomialNB()),
])

param_grid = {
    'tfidf__min_df': [20,],
    'tfidf__max_df': [.4,],
    'tfidf__ngram_range': [(1,2)],
}

grid = GridSearchCV(pipeline, param_grid, cv=kf, scoring='accuracy').fit(X_train, y)
clf = grid.best_estimator_
print(clf)
print()
print(cross_val_score(clf, X_train, y, cv=kf, scoring='accuracy').mean())

# 0.949718325755045

In [ ]:
words = {}

for word, num in clf.named_steps['tfidf'].vocabulary_.items():
    prob = clf.predict_proba([word])[0][1]
    words[word] = prob

df_topwords = pd.DataFrame([words]).transpose()
df_topwords.columns = ['probability']

df_topwords.sort_values('probability', ascending=False, inplace=True)

In [ ]:
# Lots of racism and mentions of... castration?
# People can be awful.
df_topwords

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, concatenate
from keras.layers import (CuDNNGRU, CuDNNLSTM, Bidirectional, GlobalAveragePooling1D,
                          GlobalMaxPooling1D)
from keras.preprocessing import text, sequence

In [ ]:
maxlen = 100
max_features = 30000

# Tokenize
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Sequence
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [ ]:
tokenizer.num_words

In [ ]:
def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, 100)(inp)
    x = CuDNNLSTM(64, return_sequences=True)(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model = get_model()

In [ ]:
def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, 100)(inp)
    x = CuDNNLSTM(64)(x)
    x = Dense(32, activation="relu")(x)
    outp = Dense(1, activation="sigmoid")(x)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model = get_model()

In [ ]:
batch_size = 128
epochs = 10

In [ ]:
from sklearn.model_selection import train_test_split
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y, test_size = 0.1, random_state=42)

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

es = EarlyStopping(monitor='val_acc', min_delta=.003, patience=2)
lr = ReduceLROnPlateau(monitor='val_acc', min_delta=.003, patience=1)

In [ ]:
hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                  verbose=True, callbacks=[es, lr])


y_pred = model.predict(x_test, batch_size=1024)

In [ ]:
# 50000 loss: 0.1162 - acc: 0.9545 - val_loss: 0.1052 - val_acc: 0.9582

In [ ]:
results = pd.DataFrame({"question_text": test_df["question_text"], "prediction": y_pred.flatten()})

In [ ]:
from sklearn.metrics import roc_auc_score
# y_pred_val = model.predict(X_val, batch_size=1024)
roc_auc_score(y_val, y_pred_val.flatten())

In [ ]:
# Some legitimate questions
results.sort_values('prediction').head()

In [ ]:
results[results.prediction > .4].sort_values('prediction').head()['question_text'].tolist()

In [ ]:
results[results.prediction > .5].sort_values('prediction').head()['question_text'].tolist()

In [ ]:
# Controversial, but sincere
results[results.prediction > .6].sort_values('prediction').head()['question_text'].tolist()

In [ ]:
# Questions are getting increasingly stupid...
results[results.prediction > .7].sort_values('prediction').head()['question_text'].tolist()

In [ ]:
# I'm losing IQ points just reading some of these...
results[results.prediction > .8].sort_values('prediction').sample(10)['question_text'].tolist()

In [ ]:
# See the worst of humanity (you'd rather not -- commenting this out!)
# results.sort_values('prediction', ascending=False)

In [ ]:
y_te = (y_pred[:,0] > 0.5).astype(np.int)

In [ ]:
y_te

In [ ]:
submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)

# Word2vec

In [ ]:
GLOVE_DIR = "../input/embeddings/glove.840B.300d/"

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.840B.300d.txt'), 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = ''.join(values[:-300])
        coefs = np.asarray(values[-300:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
print('Preparing embedding matrix.')

### prepare embedding matrix ###

# Use the max if there are more words than the max in our corpus
num_words = min(MAX_NB_WORDS, len(word_index))
print(num_words, 'words')
embedding_matrix = np.zeros((num_words+1, EMBEDDING_DIM))

In [ ]:
from keras.layers import (Embedding, Conv1D, MaxPooling1D,
                          Flatten, Dropout, Activation)
from keras.models import Sequential
from keras.optimizers import SGD

sgd = SGD(lr=0.0001, momentum=0.6, nesterov=True)

In [ ]:
MAX_SEQUENCE_LENGTH = 100
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 300

word_index = tokenizer.word_index

for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words+1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

x = Conv1D(256, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(3)(x)

x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(3)(x)

x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(3)(x)

x = Flatten()(x)
x = Dense(8, activation='relu')(x)
preds = Dense(1, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['acc'])

# happy learning!
model.fit(X_tra, y_tra, validation_data=(X_val, y_val),
          epochs=10, batch_size=32, verbose=1, callbacks=[lr, es])

In [ ]:
filters = 250
kernel_size = 3
hidden_dims = 250
embedding_dims = 100

model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

# happy learning!
model.fit(X_tra, y_tra, validation_data=(X_val, y_val),
          epochs=10, batch_size=32, verbose=1, callbacks=[lr, es])

In [ ]:
model.summary()